In [7]:
import pandas as pd

# sample df
df = pd.DataFrame({'id': [1, 2, 3],
                   'name': ['A', 'B', 'C'],
                   'roomno': [115, 115, 117],
                   'date_start': pd.to_datetime(['2023-04-04 08:00:00', '2023-04-04 12:00:00', '2023-04-04 08:00:00']),
                   'date_end': pd.to_datetime(['2023-04-04 14:00:00', '2023-04-04 18:00:00', '2023-04-04 14:00:00'])})

room_cap = pd.DataFrame({'roomno': [115, 117],
                         'cap': [3, 1]})

# reformat to longer
df_long = (df
           .melt(id_vars=['id', 'name', 'roomno'],
                 value_vars=['date_start', 'date_end'], 
                 var_name='what', 
                 value_name='time')
            .groupby('id'))

# df_long = df_long.groupby('id')

# define function to create date range for each group
def create_date_range(group):
    start = group['time'].min()
    end = group['time'].max()
    return pd.date_range(start=start, end=end, freq='H')

# apply function to each group and concatenate results into a single DataFrame
hourly_df = pd.concat([pd.DataFrame({'id': id,
                                     'time': create_date_range(group)}) for id, group in df_long])

# # merge base information back
hourly_df = pd.merge(hourly_df,
                     df[["id", "name", "roomno"]],
                     on='id', how='left')

# add count per hour and calculate occupancy rate
hourly_occ = (hourly_df
             .groupby(['roomno', 'time'])
             .size()
             .reset_index(name='n')
             .merge(room_cap, on='roomno')
             .assign(occ_rate=lambda x: x['n'] / x['cap']))

# print(hourly_occ)

print(hourly_occ['time'].dt.date.max())

xx = hourly_occ['time'].dt.date.max()

print(hourly_occ[(hourly_occ['time'].dt.date == xx)])


2023-04-04
